<a href="https://colab.research.google.com/github/rajeshpachar/duckdb-colab/blob/main/DuckDB_Parquet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
First we download some files and install the required libraries.

In [ ]:
!pip install pyarrow pandas
!pip install duckdb --pre --upgrade
!mkdir -p taxi
!wget https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet -O taxi/201904.parquet
!wget https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_05.parquet -O taxi/201905.parquet
!wget https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_06.parquet -O taxi/201906.parquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 50.3 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.2.1
    Uninstalling duckdb-1.2.1:
      Successfully uninstalled duckdb-1.2.1
--2025-04-09 07:20:45--  https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/duckdb/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet [following]
--2025-04-09 07:20:46--  https://github.com/duckdb/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/263853960/d481ee00-d5d4-11eb-85f7-c0e266116772?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-

In [ ]:
import pyarrow.parquet as pq
import pandas
import glob
import duckdb

# some DuckDB setup
con = duckdb.connect()
# enable automatic query parallelization
con.execute("PRAGMA threads=2")
# enable caching of parquet metadata
con.execute("PRAGMA enable_object_cache")

## Reading Multiple Parquet Files

DuckDB can read multiple parquet files using the glob syntax.

In Pandas, we need to load the files separately and concatenate them together into a single DataFrame.

In [ ]:
%%time
con.execute("SELECT pickup_at FROM 'taxi/*.parquet' LIMIT 5").df()

CPU times: user 8.88 ms, sys: 2.98 ms, total: 11.9 ms
Wall time: 9.77 ms


,pickup_at
0,2019-04-01 00:04:09
1,2019-04-01 00:22:45
2,2019-04-01 00:39:48
3,2019-04-01 00:35:32
4,2019-04-01 00:44:05


In [ ]:
%%time
con.execute("SELECT pickup_at FROM read_parquet('taxi/*.parquet') LIMIT 5").df()

CPU times: user 10.8 ms, sys: 121 µs, total: 11 ms
Wall time: 9.32 ms


,pickup_at
0,2019-04-01 00:04:09
1,2019-04-01 00:22:45
2,2019-04-01 00:39:48
3,2019-04-01 00:35:32
4,2019-04-01 00:44:05


In [ ]:
%%time
df = pandas.concat(
	[pandas.read_parquet(file)
	 for file
	 in glob.glob('taxi/*.parquet')])
print(df.head(5))

## Concatenate the three files into a single large file

As Pandas does not have native support for reading multiple files, we perform the remaining experiments on a single large file.

We use the pyarrow library to concatenate the three files into a single file.

In [ ]:
# concatenate all three parquet files into a single file
pq.write_table(pq.ParquetDataset('taxi/').read(), 'alltaxi.parquet', row_group_size=100000)

### Querying the Single File

Now let's query the single file and check the achieved performance.

In [ ]:
%%time
con.execute("SELECT * FROM 'alltaxi.parquet' LIMIT 5").df()

In [ ]:
%%time
pandas.read_parquet('alltaxi.parquet').head(5)

## Counting the Rows

Now suppose we want to figure out how many rows are in our data set. We can do that using the following code snippets.

Note that by default Pandas will read the entire Parquet file into memory again. We can manually optimize the query by specifying that only a single column should be loaded.



In [ ]:
%%time
# DuckDB
print(con.execute("SELECT COUNT(*) FROM 'alltaxi.parquet'").df())

In [ ]:
%%time
# Pandas (naive)
print(len(pandas.read_parquet('alltaxi.parquet')))

In [ ]:
%%time
# Pandas (projection pushdown)
print(len(pandas.read_parquet('alltaxi.parquet', columns=['vendor_id'])))

# Filtering Rows
It is common to use some sort of filtering predicate to only look at the interesting parts of a data set. For example, imagine we want to know how many taxi rides occur after the 30th of June 2019. We can do that using the following queries in both Pandas and DuckDB.


In [ ]:
%%time
# DuckDB
con.execute("SELECT COUNT(*) FROM 'alltaxi.parquet' WHERE pickup_at > '2019-06-30'").df()

In [ ]:
%%time
# Pandas (naive)
len(pandas.read_parquet('alltaxi.parquet')
          .query("pickup_at > '2019-06-30'"))

CPU times: user 11.5 s, sys: 9.39 s, total: 20.9 s
Wall time: 15.5 s


167022

In [ ]:
%%time
# Pandas (projection pushdown)
len(pandas.read_parquet('alltaxi.parquet', columns=['pickup_at'])
          .query("pickup_at > '2019-06-30'"))

CPU times: user 624 ms, sys: 350 ms, total: 974 ms
Wall time: 940 ms


167022

In [ ]:
%%time
# Pandas (projection + filter pushdown)
len(pandas.read_parquet('alltaxi.parquet', columns=['pickup_at'], filters=[('pickup_at', '>', '2019-06-30')]))

In [ ]:
df = pandas.read_parquet('alltaxi.parquet')

In [ ]:
%%time
# Pandas native
print(len(df[['pickup_at']].query("pickup_at > '2019-06-30'")))

## Aggregates

Now suppose we want to figure out how many rows are in our data set. We can do that using the following code snippets.

Note that by default Pandas will read the entire Parquet file into memory again. We can manually optimize the query by specifying that only a single column should be loaded.



In [ ]:
%%time
# DuckDB (SQL)
con.execute("SELECT passenger_count, COUNT(*) FROM 'alltaxi.parquet' GROUP BY passenger_count").df()

##

In [ ]:
%%time
# DuckDB (relational API)
con.from_parquet('alltaxi.parquet'
     ).aggregate('passenger_count, count(*)').df()

In [ ]:
%%time
# Pandas (naive)
pandas.read_parquet('alltaxi.parquet').groupby('passenger_count').agg({'passenger_count' : 'count'})

In [ ]:
%%time
# Pandas (projection pushdown)
pandas.read_parquet('alltaxi.parquet', columns=['passenger_count']).groupby('passenger_count').agg({'passenger_count' : 'count'})

In [ ]:
%%time
# Pandas (native)
df[['passenger_count']].groupby('passenger_count').agg({'passenger_count' : 'count'})